In [1]:
import csv
from google.colab import files

In [2]:
def write_parallel_csv(english_lines, urdu_lines, output_csv):
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['English', 'Urdu'])  # Write header

        # Write parallel sentences to CSV
        for english, urdu in zip(english_lines, urdu_lines):
            csv_writer.writerow([english.strip(), urdu.strip()])

###Files available at
https://www.kaggle.com/datasets/zainuddin123/parallel-corpus-for-english-urdu-language

In [3]:
# Upload English and Urdu text files
uploaded_english = files.upload()
uploaded_urdu = files.upload()

Saving english-corpus.txt to english-corpus.txt


Saving urdu-corpus.txt to urdu-corpus.txt


In [4]:
# Get the content of the uploaded files
english_content = list(uploaded_english.values())[0].decode('utf-8').splitlines()
urdu_content = list(uploaded_urdu.values())[0].decode('utf-8').splitlines()


In [5]:
# Write parallel sentences to CSV
output_csv_path = 'parallel_sentences.csv'
write_parallel_csv(english_content, urdu_content, output_csv_path)

print(f"CSV file '{output_csv_path}' created successfully.")

CSV file 'parallel_sentences.csv' created successfully.


In [6]:
import numpy as np
import pandas as pd
import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

<ipython-input-6-26d2d204380f>:19: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [7]:
lines=pd.read_csv("/content/parallel_sentences.csv",encoding='utf-8')

In [8]:
lines['English'].value_counts()

what is your name        4
i am divorced            4
i am at home             4
i am thirsty             3
i am sorry               3
                        ..
he is fond of fishing    1
were objective           1
he does not smoke        1
its his first time       1
i am not angry           1
Name: English, Length: 24003, dtype: int64

In [9]:
lines.head(20)

,English,Urdu
0,is zain your nephew,زین تمہارا بھتیجا ہے۔
1,i wish youd trust me,کاش تم مجھ پر بھروسہ کرتے
2,did he touch you,کیا اس نے آپ کو چھوا؟
3,its part of life,اس کی زندگی کا حصہ
4,zain isnt ugly,زین بدصورت نہیں ہے۔
5,above all be patient,سب سے بڑھ کر صبر کرو
6,i learned it from him,میں نے اسے اس سے سیکھا۔
7,why am i doing this,میں یہ کیوں کر رہا ہوں
8,i made a bad decision,میں نے ایک برا فیصلہ کیا
9,zain wont care,زین پرواہ نہیں کرے گا


In [10]:
lines = lines.dropna()
pd.isnull(lines).sum()

English    0
Urdu       0
dtype: int64

In [11]:
lines=lines[~pd.isnull(lines['English'])]

In [12]:
lines.drop_duplicates(inplace=True)

* ### Let us pick any 25000 rows from the dataset.

In [13]:
lines=lines.sample(n=24000,random_state=42)
lines.shape

(24000, 2)

In [14]:
# Lowercase all characters
lines['English']=lines['English'].apply(lambda x: x.lower())
lines['Urdu']=lines['Urdu'].apply(lambda x: x.lower())

In [15]:
# Remove quotes
lines['English']=lines['English'].apply(lambda x: re.sub("'", '', x))
lines['Urdu']=lines['Urdu'].apply(lambda x: re.sub("'", '', x))

In [16]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['English']=lines['English'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['Urdu']=lines['Urdu'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [17]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['English']=lines['English'].apply(lambda x: x.translate(remove_digits))
lines['Urdu']=lines['Urdu'].apply(lambda x: x.translate(remove_digits))

lines['Urdu'] = lines['Urdu'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['English']=lines['English'].apply(lambda x: x.strip())
lines['Urdu']=lines['Urdu'].apply(lambda x: x.strip())
lines['English']=lines['English'].apply(lambda x: re.sub(" +", " ", x))
lines['Urdu']=lines['Urdu'].apply(lambda x: re.sub(" +", " ", x))


In [18]:
# Add start and end tokens to target sequences
lines['Urdu'] = lines['Urdu'].apply(lambda x : 'START_ '+ x + ' _END')

In [19]:
lines.head()

,English,Urdu
9695,i was coughing,START_ میں کھانسی کر رہا تھا _END
283,what makes you cry,START_ آپ کو کیا رونا آتا ہے _END
3283,why are you angry with me,START_ تم مجھ سے ناراض کیوں ہو _END
8751,the floor was cold,START_ فرش ٹھنڈا تھا _END
14651,dont hurt my son,START_ میرے بیٹے کو تکلیف نہ دو _END


In [20]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['English']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_urdu_words=set()
for hin in lines['Urdu']:
    for word in hin.split():
        if word not in all_urdu_words:
            all_urdu_words.add(word)

In [21]:
len(all_eng_words)

5620

In [22]:
len(all_urdu_words)

5908

In [23]:
lines['length_eng_sentence']=lines['English'].apply(lambda x:len(x.split(" ")))
lines['length_urd_sentence']=lines['Urdu'].apply(lambda x:len(x.split(" ")))

In [24]:
lines.head()

,English,Urdu,length_eng_sentence,length_urd_sentence
9695,i was coughing,START_ میں کھانسی کر رہا تھا _END,3,7
283,what makes you cry,START_ آپ کو کیا رونا آتا ہے _END,4,8
3283,why are you angry with me,START_ تم مجھ سے ناراض کیوں ہو _END,6,8
8751,the floor was cold,START_ فرش ٹھنڈا تھا _END,4,5
14651,dont hurt my son,START_ میرے بیٹے کو تکلیف نہ دو _END,4,8


In [25]:
lines[lines['length_eng_sentence']>30].shape

(0, 4)

In [26]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_urd_sentence']<=20]

In [27]:
lines.shape

(23999, 4)

In [28]:
print("maximum length of Urdu Sentence ",max(lines['length_urd_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Urdu Sentence  20
maximum length of English Sentence  14


In [29]:
max_length_src=max(lines['length_urd_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [30]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_urdu_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_urdu_words)
num_encoder_tokens, num_decoder_tokens

(5620, 5908)

In [31]:
num_decoder_tokens += 1 #for zero padding

In [32]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [33]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [34]:
lines = shuffle(lines)
lines.head(10)

,English,Urdu,length_eng_sentence,length_urd_sentence
8597,i love this song,START_ مجھے یہ گانا بہت پسند ہے _END,4,8
1457,zain did do it,START_ زین نے کیا _END,4,5
3512,i will slap you in the face,START_ میں آپ کے منہ پر تھپڑ ماروں گا۔ _END,7,10
4279,it would be perfect,START_ یہ کامل ہو جائے گا _END,4,7
6507,lets go to a movie,START_ کسی فلم میں جانے دیں _END,5,7
24191,they will come,START_ وہ آئیں گے _END,3,5
6878,does he know me,START_ کیا وہ مجھے جانتا ہے _END,4,7
22317,we cant stay here,START_ ہم یہاں نہیں رہ سکتے _END,4,7
532,try to act your age,START_ اپنی عمر پر عمل کرنے کی کوشش کریں۔ _END,5,10
24157,i don’t wash the dishes,START_ میں برتن نہیں دھوتا _END,5,6


### Split the data into train and test

In [35]:
X, y = lines['English'], lines['Urdu']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((19199,), (4800,))

### Let us save this data

In [36]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')

In [37]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                      if t < decoder_input_data.shape[1]:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                      if t>0:
                        if t - 1 < decoder_target_data.shape[1]:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                          decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [38]:
latent_dim=300

In [39]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [40]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [41]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [42]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 input_2 (InputLayer)        [(None, None)]               0         []                            
                                                                                                  
 embedding (Embedding)       (None, None, 300)            1686000   ['input_1[0][0]']             
                                                                                                  
 embedding_1 (Embedding)     (None, None, 300)            1772700   ['input_2[0][0]']             
                                                                                              

In [43]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [44]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



Epoch 1/100


<ipython-input-44-a780c545a7a7>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),


149/149 [==============================] - 37s 163ms/step - loss: 5.3903 - val_loss: 4.7177
Epoch 2/100
149/149 [==============================] - 14s 86ms/step - loss: 4.6567 - val_loss: 4.6172
Epoch 3/100
149/149 [==============================] - 13s 84ms/step - loss: 4.5543 - val_loss: 4.5143
Epoch 4/100
149/149 [==============================] - 13s 86ms/step - loss: 4.4506 - val_loss: 4.4058
Epoch 5/100
149/149 [==============================] - 13s 88ms/step - loss: 4.3370 - val_loss: 4.3034
Epoch 6/100
149/149 [==============================] - 13s 88ms/step - loss: 4.2200 - val_loss: 4.1892
Epoch 7/100
149/149 [==============================] - 13s 87ms/step - loss: 4.0905 - val_loss: 4.0768
Epoch 8/100
149/149 [==============================] - 13s 87ms/step - loss: 3.9562 - val_loss: 3.9440
Epoch 9/100
149/149 [==============================] - 13s 87ms/step - loss: 3.8215 - val_loss: 3.8173
Epoch 10/100
149/149 [==============================] - 14s 94ms/step - loss: 3.6887

In [45]:
model.save_weights('nmt_weights.h5')

In [99]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [100]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [112]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [113]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Urdu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:len(y_train[k:k+1].values[0][6:-4])-1])

1/1 [==============================] - 0s 24ms/step
Input English sentence: what are they after
Actual Urdu Translation:  وہ کس کے بعد ہیں 
Predicted Urdu Translation:  وہ کس کے بعد ہیں


In [114]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Urdu Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Urdu Translation:', decoded_sentence[:len(y_train[k:k+1].values[0][6:-4])-1])

1/1 [==============================] - 0s 21ms/step
Input English sentence: tom would never hurt you
Actual Urdu Translation:  ٹام آپ کو کبھی تکلیف نہیں دے گا۔ 
Predicted Urdu Translation:  ٹام کبھی آپ کو تکلیف نہیں دے گا 
